The above code is importing the necessary libraries for the program.

In [ ]:
import struct
import numpy as np
import pickle

# Input Hyperparameters best practice

c 4 -p 21 --dataset sfbay_l -W 1024 -H 900 

c 4 -p 21 --dataset sfbay_c -W 1426 -H 1876 

c 6 -p 7 --dataset flevo_l -W 1024 -H 750

c 6 -p 21 --dataset flevo_c -W 1639 -H 2393

# Enter input <br>
* **dataset choose from [sfbay_l, sfbay_c, flevo_l, or flevo_c]**<br>
* **channels number of channels choose form 3 or 4 or 6**<br>
* **patch sliding window patch size**<br>
* **W  width size of the scene**<br>
* **H  height of the scene**

# Input Function 

In [ ]:
def Data_entry ():
    """
    This function takes the name of the dataset, the number of channels, the window size, the width and
    the height of the image as input and returns the name of the dataset, the number of channels, the
    window size, the width and the height of the image and the file size
    """
    args={} 
    args["data"]=input("please enter dataset name \n")
    args["channels"]=int(input("Please enter number of channels \n"))
    
    args["patch"]=int(input("please enter window size \n" ))
    args["W"]=int(input("please enter width of the image \n"))
    args["H"]=int(input("please enter hight of the image \n"))
    filesize = args["W"] * args["H"]
    return args["data"],args["channels"],args["patch"],args["W"],args["H"],filesize

Defining Dataset with it's Hyperparameters

In [ ]:
dataset,channels,patch_size,m_W,m_H,filesize=Data_entry ()

In [ ]:
def read_row(filename):
    """
    It reads a binary file, and returns a numpy array of the data
    
    :param filename: the name of the file to be read
    :return: the data from the file in the form of an array.
    """
    rdata = np.zeros(filesize, dtype = 'float32')
    f = open(filename, 'rb')
    for l in range(0, +filesize):
        (num,) = struct.unpack('f', f.read(4))
        rdata[l] = num
    return rdata

# Reading the data from the binary files and saving it in a pickle file

In [ ]:
row_data = np.zeros((filesize, channels), dtype = 'float32')
if channels == 3:
    filenames = ['C11dbs.bin', 'C22dbs.bin', 'C33dbs.bin']#['T11dbs.bin', 'T22dbs.bin', 'T33dbs.bin']
elif channels == 4:
    filenames = ['T11dbs.bin', 'T22dbs.bin', 'T33dbs.bin', 'span_dbs.bin']

In [ ]:
elif channels == 5:
    filenames = ['T11dbs.bin', 'T22dbs.bin', 'T33dbs.bin', 'C11dbs.bin', 'C22dbs.bin']
    
elif channels == 6:
    filenames = ['T11dbs.bin', 'C11dbs.bin' ,'T22dbs.bin','C22dbs.bin', 'T33dbs.bin',  'C33dbs.bin']

In [ ]:
for i in range(0, channels):
    row_data[:, i] = read_row(dataset+ '/' + filenames[i])

# Saving the objects:

In [ ]:
with open(dataset + '/' + dataset + '_' + str(channels) + '_rowdata.pkl', 'wb') as f:
    pickle.dump([row_data], f, protocol = 4)

Defining the size of the training and testing data.

In [ ]:
if dataset == 'sfbay_l':
    train_size = 1462 # SFBay_L
    test_size = 121997 # SFBay_L
elif dataset == 'sfbay_c':
    train_size = 2500 # SFBay_C
    test_size = 250000 # SFBay_C
elif dataset == 'flevo_l':
    train_size = 4211 # Flevo_L
    test_size = 204023 # Flevo_L
elif dataset == 'flevo_c':
    train_size = 2000 # Flevo_C
    test_size = 200000 # Flevo_C

# Reading the data from the two files, and returns the data as two numpy arrays

In [ ]:
def read_gtd(filename, size):
    """
    It reads in the data from the two files, and returns the data as two numpy arrays
    
    :param filename: the name of the file you want to read
    :param size: the number of data points in the file
    :return: the positions and labels of the data.
    """
    positions = np.zeros((size,), dtype = 'float32')
    labels = np.zeros((size,), dtype = 'float32')
    f1 = open(filename + '_positions.gtd', 'rb')
    f2 = open(filename + '_labels.gtd', 'rb')
    for l in range(0, size):
        (num1,) = struct.unpack('f', f1.read(4))
        (num2,) = struct.unpack('f', f2.read(4))
        positions[l] = num1
        labels[l] = num2
    return positions, labels

In [ ]:
train_positions, train_labels = read_gtd(dataset + '/' + dataset + '_train', train_size)
test_positions, test_labels = read_gtd(dataset+ '/' + dataset + '_test', test_size)
#Converting from float to int
train_positions = np.array(train_positions, dtype = 'int')
test_positions = np.array(test_positions, dtype = 'int')
train_labels = np.array(train_labels, dtype = 'int')
test_labels = np.array(test_labels, dtype = 'int')

Loading the data from the pickle file.

In [ ]:
with open(dataset + '/' + dataset + '_' + str(channels) + '_rowdata.pkl', 'rb') as f:
    data = pickle.load(f)

Reshaping the data to the size of the image.

In [ ]:
data = np.array(data[0])
data = np.reshape(data, [m_H, m_W, channels])

In [ ]:
data_padded = np.zeros([m_H + patch_size - 1, m_W + patch_size - 1, channels], dtype='float32')

Padding the data with zeros.

In [ ]:
startid = (patch_size - 1) // 2
data_padded[startid:startid + m_H, startid:startid + m_W, :] = data

In [ ]:
print(data_padded.shape)

Creating a patch of the image.

In [ ]:
patch_data = np.zeros([filesize, patch_size, patch_size, channels], dtype = 'float32')

In [ ]:
index = 0
for i in range(startid, startid + m_H):
    for j in range(startid, startid + m_W):
        for c in range(0, channels):
            patch_data[index, :, :, c] = data_padded[i - startid : i + startid + 1, j - startid : j + startid + 1, c]
        index = index + 1
print('Processed: ', (index/filesize)*100)

Checking  patch data shape

In [ ]:
print( patch_data.shape)

In [ ]:
print(patch_data[0].shape)

Saving the patch data in a pickle file.

In [ ]:
with open(dataset + '/' + dataset + '_' + str(channels) + '_' + str(patch_size) + '_' + 'patchdata' + '.pkl', 'wb') as f:
    pickle.dump(patch_data, f, protocol = 4)

Creating a matrix of zeros with the size of the training and testing data.

In [ ]:
train_data = np.zeros([train_size, patch_size, patch_size, channels], dtype = 'float32')
test_data = np.zeros([test_size, patch_size, patch_size, channels], dtype = 'float32')
train_data = patch_data[train_positions]
test_data = patch_data[test_positions]

Saving the data in a pickle file.

In [ ]:
with open(dataset + '/' + dataset + '_' + str(channels) + '_' + str(patch_size) +'.pkl', 'wb') as f:
    pickle.dump([train_data, train_labels, test_data, test_labels], f, protocol = 4)

# Change  labels of flevo_l

Loading the data from the pickle file.

In [ ]:
dataset,channels,patch_size,m_W,m_H,filesize=Data_entry ()

In [ ]:
with open(dataset  + '/' + dataset + '_' + str(channels) + '_' + str(patch_size) +'.pkl', 'rb') as f:
       x_train, y_train,  x_test, y_test = pickle.load(f)

In [ ]:
#checking for the number of classes in trainning dataset
unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
len(y_train)

Changing the labels of the training data.

In [ ]:
cropland = [2,4,5,6,7,8,10,11,12,13]
label_new_train = [] 
for i in y_train:
    if i == 3:
        i = 1
    elif i in cropland:
        i = 3
    elif i == 14 or i == 9:
        i = 2
    label_new_train.append(i)

In [ ]:
#checking for the number of classes after converting classes
unique, counts = np.unique(label_new_train, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
label_new_train = np.array(label_new_train)
label_new_train.shape

In [ ]:
#checking for the number of classes in testing Dataset
unique, counts = np.unique(y_test, return_counts=True)
print(np.asarray((unique, counts)).T)

Changing the labels of the testing data.

In [ ]:
cropland = [2,4,5,6,7,8,10,11,12,13]
label_new_test = [] 
for i in y_test:
    if i == 3:
        i = 1
    elif i in cropland:
        i = 3
    elif i == 14 or i == 9:
        i = 2
    label_new_test.append(i)

In [ ]:
#checking for the number of classes after converting classes
unique, counts = np.unique(label_new_test, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
label_new_test = np.array(label_new_test)
label_new_test.shape

In [ ]:
# Saving the data in a pickle file.
with open(dataset + '/' + dataset + '_' + str(channels) + '_' + str(patch_size) + '_' + '4-classes' + '.pkl', 'wb') as f:
    pickle.dump([train_data, label_new_train, test_data, label_new_test], f, protocol = 4)

Importing the necessary libraries for the program.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import cv2

# Showing the image of the dataset.

In [ ]:
df2 = pd.read_pickle(f"{dataset}/{dataset}_{str(channels)}_rowdata.pkl")
df2 = df2[0]
df2 = np.reshape(df2, [m_H, m_W, channels])

In [ ]:
cv2.namedWindow("image", cv2.WINDOW_NORMAL)

In [ ]:
cv2.imshow('image', df2[:,:,0])

In [ ]:
cv2.waitKey(0) 

In [ ]:
cv2.destroyAllWindows() 